# Running a case with tracers

DALES can handle an arbitrary number of tracers. Examples of tracers are aerosols, hydrometeors, chemical species, et cetera. We distinguish two types of tracers:

- Internal tracers. These are tracers defined internally, by the model itself. For example, the the bulkmicrophysics schemes define various tracers that represent quantities like rain and cloud droplets. The user does not have to provide any input for these tracers.
- User-defined tracers. These tracers need to be provided by the user in a NetCDF file. 

## Preparing the NetCDF file

First, we have to define a new tracer NetCDF file. This file has to be called `tracers.<iexpnr>.nc`, where `<iexpnr>` corresponds to what you have set as `iexpnr` in the `&RUN` section of your namelist.

In [7]:
import netCDF4 as nc

tracers_nc = nc.Dataset("tracers.001.nc", "w")

Next, we'll create a dimension corresponding to the vertical levels. Tracers are defined at the full levels (cell centers), so the dimension we have to provide is `zt`. We don't have to provide the actual values of the levels, since those have already been read from `init.001.nc`. This example is based on the benchmark RICO case, which uses 126 vertical levels.

In [8]:
height_dim = tracers_nc.createDimension("zt", size=126)
print(tracers_nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): zt(126)
    variables(dimensions): 
    groups: 


## Adding a tracer
Now we can define our tracer. For this example, we make a tracer that represents carbon dioxide (CO2): 

In [9]:
co2 = tracers_nc.createVariable("co2", "f8", ("zt",))
print(tracers_nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): zt(126)
    variables(dimensions): float64 co2(zt)
    groups: 


And we provide some (fake) initial profile:

In [ ]:
import numpy as np

co2[:] = np.ones(126) * 1e-6

## Tracer attributes
We can provide a number of attributes for our new tracer. These attributes either provide extra information for output, or enable extra schemes for the tracer.

| Attribute    | Type    | Description |
| ------------ | ------- | ----------- |
| `long_name`  | String  | Full name of the tracer |
| `unit`       | String  | Unit of the tracer |
| `molar_mass` | Float   | Molar mass of the tracer |
| `lemis`      | Integer | Tracer is emitted (1=True, 0=False)|
| `lreact`     | Integer | Tracer is involved in chemistry |
| `ldep`       | Integer | Tracer is deposited |
| `lags`       | Integer | Tracer is involved in photosynthesis |

Attributes can be set like this:

In [13]:
co2.long_name = "carbon dioxide"
co2.unit = "kg/kg"
co2.molar_mass = "44.009"
co2.lemis = 1
co2.lags = 1

print(co2)

RuntimeError: NetCDF: Not a valid ID

Don't forget to close the NetCDF file:

In [10]:
tracers_nc.close()

We can check the end result with `ncdump`:

In [12]:
!ncdump tracers.001.nc

netcdf tracers.001 {
dimensions:
	zt = 126 ;
variables:
	double co2(zt) ;
data:

 co2 = _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, 
    _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, 
    _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, 
    _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, 
    _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, 
    _, _, _, _, _, _, _ ;
}
